# Conversational data exploration tutorial

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensein/senselab/blob/main/tutorials/audio/conversational_data_exploration.ipynb)


In this tutorial we demonstrate how to perform **conversational data exploration** using the `senselab` library.  The goal is to load raw audio samples, prepare them for analysis, perform speaker diarization (to understand who speaks and when) transcribe speech segments (to learn what has been said), extract acoustic features, speaker embeddings and self-supervised model embeddings (to describe each speech segment), and finally assemble the results into a convenient JSON format for further analysis of different kinds.

We will make use of the following `senselab` utilities:

* `read_audios` – load audio files from disk into `Audio` objects;
* `downmix_audios_to_mono` and `resample_audios` – convert the audio to mono and a common sampling rate (16 kHz);
* `diarize_audios` – run the NVIDIA diarization model `nvidia/diar_sortformer_4spk-v1` to label each speaker in the conversation;
* `extract_segments` – cut the audio into segments based on diarization timestamps;
* `transcribe_audios_with_transformers` – perform automatic speech recognition (ASR) using the model `openai/whisper-tiny` and `openai/whisper-small`;
* `extract_features_from_audios` – compute OpenSMILE, Praat/Parselmouth and SQUIM features for each segment;
* `extract_speaker_embeddings_from_audios` – compute a fixed‑length speaker embedding for each segment using `speechbrain/spkrec-ecapa-voxceleb` and `speechbrain/spkrec-resnet-voxceleb`;
* `extract_ssl_embeddings_from_audios` – compute time-pooled fixed‑length self-supervised model embedding for each segment using `microsoft/wavlm-base`.


The final output will be a list of JSON objects, one per speaker turn, containing the transcript, extracted features, embeddings and the speaker identifier. We intend this as a starting point for further analyses. 

## Installation

In [1]:
%pip install 'senselab[audio]'

Note: you may need to restart the kernel to use updated packages.


## Data download

The example we use in this case has been automatically generated using Higgs audio v2 by Boson AI. See [here](https://huggingface.co/spaces/smola/higgs_audio_v2) for details.

In [2]:
!mkdir -p tutorial_audio_files
!wget -O tutorial_audio_files/english_conversation_higgs_audio_v2.wav https://github.com/sensein/senselab/raw/main/src/tests/data_for_testing/english_conversation_higgs_audio_v2.wav

--2025-09-15 19:25:13--  https://github.com/sensein/senselab/raw/main/src/tests/data_for_testing/english_conversation_higgs_audio_v2.wav
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sensein/senselab/main/src/tests/data_for_testing/english_conversation_higgs_audio_v2.wav [following]
--2025-09-15 19:25:14--  https://raw.githubusercontent.com/sensein/senselab/main/src/tests/data_for_testing/english_conversation_higgs_audio_v2.wav
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1031084 (1007K) [audio/wav]
Saving to: ‘tutorial_audio_files/english_conversation_higgs_audio_v2.wav’

tutorial_audio_fi

## Processing

In [3]:
# Necessary libraries
import json
import os

import torch

# Audio data structure
from senselab.audio.data_structures import Audio

# Feature extraction API
from senselab.audio.tasks.features_extraction import extract_features_from_audios
from senselab.audio.tasks.input_output.utils import read_audios

# Pre‑processing functions
from senselab.audio.tasks.preprocessing.preprocessing import (
    downmix_audios_to_mono,
    extract_segments,
    resample_audios,
)

# Speaker diarization API
from senselab.audio.tasks.speaker_diarization import diarize_audios

# Speaker embeddings extraction API
from senselab.audio.tasks.speaker_embeddings import extract_speaker_embeddings_from_audios

# Automatic Speech Recognition API
from senselab.audio.tasks.speech_to_text import transcribe_audios

# Self-supervised embeddings extraction API
from senselab.audio.tasks.ssl_embeddings import extract_ssl_embeddings_from_audios

# Utility classes for specifying models and devices
from senselab.utils.data_structures import HFModel, SpeechBrainModel

/Users/fabiocat/Library/Caches/pypoetry/virtualenvs/senselab-KP8v1V64-py3.12/lib/python3.12/site-packages/torchaudio/pipelines/_squim_pipeline.py:53: UserWarning: torchaudio.utils.download.download_asset has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  path = torchaudio.utils.download_asset(f"models/{self._path}")
/Users/fabiocat/Library/Caches/pypoetry/virtualenvs/senselab-KP8v1V64-py3.12/lib/python3.12/site-packages/torchaudio/pipelines/_squim_pipeline.py:128: UserWarning: torchaudio.utils.download.download_asset has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  path = torchaudio.utils.downlo

In [4]:
# Note: it's recommend to use absolute paths for file operations
# to avoid issues with how Pydra (a Python dependency used under the hood) caches and retrieves files
file_paths = [os.path.abspath("tutorial_audio_files/english_conversation_higgs_audio_v2.wav")]
print("Audio files to process:", file_paths)

Audio files to process: ['/Users/fabiocat/git/senselab/tutorials/audio/tutorial_audio_files/english_conversation_higgs_audio_v2.wav']


In [5]:
# Load the audio files into senselab `Audio` objects.  Each Audio object lazily loads.
# Use `plugin="cf"` to specify the use of concurrent futures for parallel processing.
audios = read_audios(file_paths, plugin="cf")

In [6]:
# Speaker diarization and ASR models typically expect a single channel and a 16kHz sampling rate.

# Convert the audio files to mono. The `downmix_audios_to_mono` function averages multiple channels.
# Use `plugin="cf"` to specify the use of concurrent futures for parallel processing.
audios = downmix_audios_to_mono(audios, plugin="cf")

# Resample the audio to the target sampling rate required by downstream models.
# Use `plugin="cf"` to specify the use of concurrent futures for parallel processing.
target_sr = 16000
audios = resample_audios(audios, resample_rate=target_sr, plugin="cf")


/Users/fabiocat/git/senselab/src/senselab/audio/data_structures/audio.py:183: UserWarning: torchaudio._backend.utils.info has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  info = torchaudio.info(filepath)
/Users/fabiocat/Library/Caches/pypoetry/virtualenvs/senselab-KP8v1V64-py3.12/lib/python3.12/site-packages/torchaudio/_backend/soundfile_backend.py:120: UserWarning: torchaudio._backend.common.AudioMetaData has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pyt

In [7]:
# Perform speaker diarization on the pre‑processed audio.
# We use the NVIDIA Sortformer model from Hugging Face.  You can change the model path
# if you want to experiment with other diarization models.
# "nvidia/diar_sortformer_4spk-v1" is generally pretty good, but can only detect up to 4 speakers
diar_model = HFModel(path_or_uri="nvidia/diar_sortformer_4spk-v1")

# Run the diarization.  The output is a list (one per audio file) of lists of
# `ScriptLine` objects.  Each ScriptLine has a `speaker` label and start/end times (in seconds).
diarization_results = diarize_audios(
    audios=audios,
    model=diar_model,
)

[NeMo W 2025-09-15 19:25:52 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    num_spks: 4
    session_len_sec: 90
    soft_label_thres: 0.5
    soft_targets: false
    labels: null
    batch_size: 4
    shuffle: true
    num_workers: 18
    validation_mode: false
    use_lhotse: false
    use_bucketing: false
    num_buckets: 10
    bucket_duration_bins:
    - 10
    - 20
    - 30
    - 40
    - 50
    - 60
    - 70
    - 80
    - 90
    pin_memory: true
    min_duration: 80
    max_duration: 90
    batch_duration: 400
    quadratic_duration: 1200
    bucket_buffer_size: 20000
    shuffle_buffer_size: 10000
    window_stride: 0.01
    subsampling_factor: 8
    
[NeMo W 2025-09-15 19:25:52 nemo_logging:405] If you intend to do validation, please call the ModelPT.setup_validation

[NeMo I 2025-09-15 19:25:52 nemo_logging:393] PADDING: 16
[NeMo I 2025-09-15 19:25:52 nemo_logging:393] Model SortformerEncLabelModel was successfully restored from /Users/fabiocat/.cache/huggingface/hub/models--nvidia--diar_sortformer_4spk-v1/snapshots/a036d2486ba4afec9ad9dc0689a775428e9b7755/diar_sortformer_4spk-v1.nemo.


2025-09-15 19:25:52,860 - senselab - INFO - Time taken to initialize the NVIDIA Sortformer model: 2.11 seconds


[NeMo I 2025-09-15 19:25:52 nemo_logging:393] No postprocessing YAML file has been provided. Default postprocessing configurations will be applied.


Diarizing: 100%|██████████| 1/1 [00:00<00:00,  1.39it/s]
2025-09-15 19:25:53,593 - senselab - INFO - Time taken to perform diarization: 0.73 seconds


In [8]:
# Build a list of (Audio, list_of_segment_times) pairs.  
from typing import List, Tuple

# Each entry corresponds to one source audio file and contains all diarization intervals for that file. 
# We convert the ScriptLine objects into (start, end) tuples.
segments_info = []
for audio, script_lines in zip(audios, diarization_results):
    times: List[Tuple[float, float]] = []
    for line in script_lines:
        times.append((line.start, line.end))
    segments_info.append((audio, times))

# Use `extract_segments` to slice the audio into individual speaker turns.  This returns
# a list (one per audio file) of lists of `Audio` segments.  Each segment retains the same
# sampling rate and metadata as the resampled recording.
segmented_audios_list = extract_segments(segments_info)

In [9]:
# Populate metadata from file_paths (per file) and diarization_results (per segment)
for i, (segments, lines) in enumerate(zip(segmented_audios_list, diarization_results)):
    src_path = file_paths[i] if i < len(file_paths) else None
    if len(segments) != len(lines):
        print(f"[warn] file #{i}: {len(segments)} segments vs {len(lines)} diarization entries — zipping to shortest.")
    n = min(len(segments), len(lines))
    for j in range(n):
        seg = segments[j]
        line = lines[j]

        # choose the container
        md = seg.metadata

        # fill fields from order-aligned sources
        md["original_file"] = src_path
        md["sampling_rate"] = seg.sampling_rate
        md["speaker_id"] = (
            getattr(line, "speaker", None)
        )
        md["segment_start"] = (
            getattr(line, "start", None)
        )
        md["segment_end"] = (
            getattr(line, "end", None)
        )

# Flatten to a single list of Audio objects
flattened_segments = [seg for group in segmented_audios_list for seg in group]

In [10]:
# Transcribe the segments using the OpenAI Whisper ASR models.
asr_whisper_tiny_model = HFModel(path_or_uri="openai/whisper-tiny")
asr_whisper_small_model = HFModel(path_or_uri="openai/whisper-small")

asr_whisper_tiny_out = transcribe_audios(audios=flattened_segments, model=asr_whisper_tiny_model)
asr_whisper_small_out = transcribe_audios(audios=flattened_segments, model=asr_whisper_small_model)

for seg, tiny, small in zip(flattened_segments, asr_whisper_tiny_out, asr_whisper_small_out):
    seg.metadata["transcripts"] = {
        "openai/whisper-tiny": tiny.text,
        "openai/whisper-small": small.text
    }

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).
2025-09-15 19:25:56,549 - senselab - INFO - Time taken to initialize the hugging face ASR pipeline: 1.97 seconds
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate yo

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).
2025-09-15 19:26:00,626 - senselab - INFO - Time taken to initialize the hugging face ASR pipeline: 1.74 seconds
2025-09-15 19:26:15,945 - senselab - INFO - Time taken for transcribing the audios: 15.32 seconds


In [11]:
# Extract features from the audio segments
feat_out = extract_features_from_audios(
    audios=flattened_segments,
    opensmile=True,
    parselmouth=True,
    torchaudio=False,
    torchaudio_squim=True
)
for seg, feats in zip(flattened_segments, feat_out):
    seg.metadata["features"] = feats


In [12]:
# Extract speaker embeddings from the audio segments
spk_ecapatdnn_model = SpeechBrainModel(path_or_uri="speechbrain/spkrec-ecapa-voxceleb")
spk_resnet_model = SpeechBrainModel(path_or_uri="speechbrain/spkrec-resnet-voxceleb")

spk_ecapatdnn_out = extract_speaker_embeddings_from_audios(
    audios=flattened_segments,
    model=spk_ecapatdnn_model
)
spk_resnet_out = extract_speaker_embeddings_from_audios(
    audios=flattened_segments,
    model=spk_resnet_model
)

for seg, emb_ecapa, emb_resnet in zip(flattened_segments, spk_ecapatdnn_out, spk_resnet_out):
    seg.metadata["speaker_embeddings"] = {
        "speechbrain/spkrec-ecapa-voxceleb": emb_ecapa.tolist(),
        "speechbrain/spkrec-resnet-voxceleb": emb_resnet.tolist()
    }

In [13]:
# Extract self-supervised learning (SSL) embeddings from the audio segments
wavlm_model = HFModel(path_or_uri="microsoft/wavlm-base")

wavlm_ssl_out = extract_ssl_embeddings_from_audios(
    audios=flattened_segments,
    model=wavlm_model
)

mean_pooled_wavlm_ssl_out = [torch.mean(tensor, dim=1) for tensor in wavlm_ssl_out]
for seg, emb in zip(flattened_segments, mean_pooled_wavlm_ssl_out):
    seg.metadata["ssl_embeddings"] = {
        "microsoft/wavlm-base": emb.squeeze().tolist(),
    }

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
def build_json_from_segment(seg: Audio) -> object:
    """Build a JSON object from a segment's metadata.
    
    Args:
        seg: The audio segment to build the JSON object from.

    Returns:
        A JSON object representing the segment's metadata.
    """
    md = getattr(seg, "metadata", {}) or {}

    # transcripts
    transcripts = md.get("transcripts")

    # speaker embeddings
    speaker_embeddings = md.get("speaker_embeddings")

    # self-supervised embeddings
    ssl_embeddings = md.get("ssl_embeddings")

    # features
    features = md.get("features")

    # times
    segment_start = md.get("segment_start")
    segment_end = md.get("segment_end")

    # sampling rate
    segment_sampling_rate = md.get("sampling_rate")

    return {
        "original_file": md.get("original_file"),
        "sampling_rate": segment_sampling_rate,
        "speaker_id": md.get("speaker_id"),
        "start": segment_start,
        "end": segment_end,
        "transcripts": transcripts,
        "features": features,
        "speaker_embeddings": speaker_embeddings,
        "ssl_embeddings": ssl_embeddings
    }

# Build the list of JSON objects (one per segment)
output_json = [build_json_from_segment(seg) for seg in flattened_segments]

# Optional: pretty-print first item
if output_json:
    print(json.dumps(output_json[0], indent=2))

{
  "original_file": "/Users/fabiocat/git/senselab/tutorials/audio/tutorial_audio_files/english_conversation_higgs_audio_v2.wav",
  "sampling_rate": 16000,
  "speaker_id": "speaker_0",
  "start": 0.0,
  "end": 3.28,
  "transcripts": {
    "openai/whisper-tiny": "I can't believe you did that without even asking me first.",
    "openai/whisper-small": "I can't believe you did that without even asking me first!"
  },
  "features": {
    "opensmile": {
      "F0semitoneFrom27.5Hz_sma3nz_amean": 42.10844039916992,
      "F0semitoneFrom27.5Hz_sma3nz_stddevNorm": 0.08829638361930847,
      "F0semitoneFrom27.5Hz_sma3nz_percentile20.0": 38.28285217285156,
      "F0semitoneFrom27.5Hz_sma3nz_percentile50.0": 42.84529495239258,
      "F0semitoneFrom27.5Hz_sma3nz_percentile80.0": 44.90517807006836,
      "F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2": 6.622325897216797,
      "F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope": 60.14593505859375,
      "F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope": 34.8432